> Routing allows you to create non-deterministic chains where the output of a previous step defines the next step. <br>
> Routing helps provide structure and consistency around interactions with LLMs.<br>
> `Routing` <font color=blue>允许你创建一个 非确定性 链, 其中上一步的输出定义下一步</font><br>
> `Routing` <font color=blue>提供与LLM交互的结构和一致性</font><br>

# Sequential

> 1. The next step after calling a language model is to make a series of calls to a language model.<br>
     调用LLM之后的步骤是 一些列的调用LLM<br>
> 2. This is particularly useful when you want to take the output from one call and use it as the input to another.<br>
      这在将一个LLM的输出用于另一个LLM的输入. 注意这个不是Router的分支而是一个解一个

> The recommended way to do this is using the LangChain Expression Language. 
> The legacy way is using the `SequentialChain`, which we continue to document here for backwards compatibility.

> As a toy example, let's suppose we want to create a chain that first creates a play synopsis and then generates a play review based on the synopsis.<br>
> 链首先创建一个戏剧概要, 然后根据概要生成戏剧评论

In [3]:
from langchain.prompts import PromptTemplate

# 剧作家
synopsis_prompt = PromptTemplate.from_template(
    """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.

Title: {title}
Playwright: This is a synopsis for the above play:"""
)

# 评论家
review_prompt = PromptTemplate.from_template(
    """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
)

## Using LCEL

> Creating a sequence of calls (to LLMs or any other component/arbitrary function) is precisely what LangChain Expression Language was designed for.

> 创建一系列调用（对 LLM 或任何其他组件/任意函数）正是 LangChain 表达式语言的设计目的

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser

In [5]:
llm = ChatOpenAI()

In [6]:
chain = ( 
    {"synopsis": synopsis_prompt | llm | StrOutputParser()}
    | review_prompt
    | llm
    | StrOutputParser()
)

In [7]:
chain.invoke({"title": "Tragedy at sunset on the beach"})
# o - o : 海滩日落时的悲剧

'"Tragedy at Sunset on the Beach" is a mesmerizing and thought-provoking play that immerses its audience in a world of raw emotions and complex characters. Set against the backdrop of a serene beach at sunset, this production offers a visually stunning experience that enhances the dramatic tension and adds depth to the storytelling.\n\nThe play\'s strength lies in its ability to capture the intricacies of human relationships and the profound impact they can have on our lives. The characters are skillfully crafted, each with their own personal struggles and desires. Sarah, portrayed with remarkable vulnerability, serves as the emotional center of the story, grappling with her tragic past and seeking solace in the beauty of the beach. Michael, the reclusive artist, brings a quiet intensity to the stage, finding inspiration in the colors of the sunset. Their connection is beautifully portrayed, and the chemistry between the actors is palpable.\n\nHowever, it is the arrival of the boistero

> If we wanted to get back the synopsis as well we could do:<br>
> 如果我们想取回概要, 可以这样做

In [9]:
from langchain.schema.runnable import RunnablePassthrough

synopsis_chain = synopsis_prompt | llm | StrOutputParser()
review_chain = review_prompt | llm | StrOutputParser()
chain = {"synopsis": synopsis_chain} | RunnablePassthrough.assign(review=review_chain)
chain.invoke({"title": "Tragedy at sunset on the beach"})

{'synopsis': '"Tragedy at Sunset on the Beach" is a gripping and emotional play that explores the complexities of love, loss, and the fragility of human existence. Set against the breathtaking backdrop of a picturesque beach at sunset, the story unfolds with a group of diverse characters whose lives intertwine in unexpected ways.\n\nThe play revolves around Sarah, a young woman haunted by a tragic past, who seeks solace and escape in the serenity of the beach at sunset. As she gazes out at the horizon, she encounters Michael, a charming but mysterious stranger who seems to be harboring his own secrets.\n\nTheir chance meeting sparks a whirlwind romance, filled with passion and intensity. However, as the sun sets on their love story, a series of unforeseen events and revelations plunge them into a downward spiral of heartbreak and despair. Secrets are exposed, betrayals are revealed, and the once idyllic beach becomes a haunting symbol of their shattered dreams.\n\nAs the play progresse

## Legacy SequentialChain

Sequential chains allow you to connect multiple chains and compose them into <font color=blue>pipelines</font> that execute some specific scenario. There are two types of sequential chains:

* `SimpleSequentialChain`: The simplest form of sequential chains, where each step has a singular input/output, and the output of one step is the input to the next.<br>
                           其中每个步骤都有一个单一的输入/输出，并且一个步骤的输出是下一步的输入。

* `SequentialChain` : A more general form of sequential chains, allowing for multiple inputs/outputs.<br>
                      顺序链的更通用形式，允许多个输入/输出

### SimpleSequentialChain

In [10]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# This is an LLMChain to write a synopsis given a title of a play.
llm = OpenAI(temperature=0.7)
synopsis_chain = LLMChain(llm=llm, prompt=synopsis_prompt)

In [11]:
# This is an LLMChain to write a review of a play given a synopsis.
llm = OpenAI(temperature=0.7)
review_chain = LLMChain(llm=llm, prompt=review_prompt)

In [12]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SimpleSequentialChain

overall_chain = SimpleSequentialChain(
    chains=[synopsis_chain, review_chain], verbose=True
)

review = overall_chain.run("Tragedy at sunset on the beach")



> Entering new SimpleSequentialChain chain...


Tragedy at Sunset on the Beach is a powerful drama that follows the story of two lifelong friends, Sarah and James. After many years of not seeing each other, the two are reunited on a beach at sunset. As they rekindle their friendship, secrets and past traumas are revealed that threaten to tear them apart. Through this tragedy, Sarah and James must confront the truth about their pasts and question what really matters in life. Along the way, they must also learn to forgive each other and find a way to move forward.


Tragedy at Sunset on the Beach is a powerful drama that presents a thought-provoking story of two lifelong friends. The reunion of Sarah and James on a beach at sunset sets the stage for a journey of reflection and understanding. The play reveals secrets and past traumas that serve to test their friendship and challenge the audience to reflect on their own beliefs and experiences. The script is well-crafted, and the actors 

In [13]:
print(review)



Tragedy at Sunset on the Beach is a powerful drama that presents a thought-provoking story of two lifelong friends. The reunion of Sarah and James on a beach at sunset sets the stage for a journey of reflection and understanding. The play reveals secrets and past traumas that serve to test their friendship and challenge the audience to reflect on their own beliefs and experiences. The script is well-crafted, and the actors deliver an emotional performance that is both heartbreaking and inspiring. It is a powerful story of resilience and forgiveness, and an excellent reminder of how life’s experiences can shape our beliefs and relationships. Tragedy at Sunset on the Beach is a must-see production that offers a meaningful and captivating journey.


### SequentialChain

> Of course, not all sequential chains will be as simple as passing a single string as an argument and getting a single string as output for all steps in the chain. <br>
> 当然并不是所有的都是`一步一步来的`

> Of particular importance is how we name the input/output variables.<br>
> 特别重要的是我们<font color=blue size=4>如何命名输入/输出变量</font>

In [14]:
# This is an LLMChain to write a synopsis given a title of a play and the era it is set in.
llm = OpenAI(temperature=0.7)

synopsis_template = """You are a playwright. Given the title of play and the era it is set in, it is your job to write a synopsis for that title.

Title: {title}
Era: {era}
Playwright: This is a synopsis for the above play:"""

synopsis_prompt_template = PromptTemplate(
    input_variables=["title", "era"], template=synopsis_template
)

synopsis_chain = LLMChain(
    llm=llm, prompt=synopsis_prompt_template, output_key="synopsis"
)

In [15]:
# This is an LLMChain to write a review of a play given a synopsis.
llm = OpenAI(temperature=0.7)
template = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:"""
prompt_template = PromptTemplate(input_variables=["synopsis"], template=template)
review_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="review")

# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[synopsis_chain, review_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["synopsis", "review"],
    verbose=True,
)

# era: 维多利亚 英格兰时代
overall_chain({"title": "Tragedy at sunset on the beach", "era": "Victorian England"})



> Entering new SequentialChain chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'synopsis': "\n\nTragedy at Sunset on the Beach is set in Victorian England, and follows the story of two star-crossed lovers, Marie and Edward. They come from two very different backgrounds - Marie is a poor fisherman's daughter, and Edward is the son of a wealthy aristocrat. Despite their class differences, they fall deeply in love and plan to elope. \n\nOn the night that they plan to leave, Edward and Marie meet on the beach at sunset for one final farewell. Marie is unaware that Edward has been disowned by his family and is penniless; his only hope is to marry a wealthy heiress. Marie overhears Edward making plans to meet with the heiress on the same night and is heartbroken. In her despair, she runs into the sea and drowns. \n\nEdward is devastated by Marie's death and is left with a heavy heart and a broken spirit. He soon discovers that the heiress he was to meet was a trap set by his family, who sought to

> 多输出? 输出应该是唯一的 <br>
> 输入变量是多个, 但并不是多个`链图`

### Memory in Sequential Chains

> Sometimes you may want to pass along some context to use in each step of the chain or in a later part of the chain, but maintaining and chaining together the input/output variables can quickly get messy. Using SimpleMemory is a convenient way to do manage this and clean up your chains.

> 希望传递上下文给chain的每一个步骤, 或者 后续部分的chain, 但是会混乱, 所以使用 `SimpleMemory` 是很方便的

> For example, using the previous playwright SequentialChain, lets say you wanted to include some context about date, time and location of the play, and using the generated synopsis and review, create some social media post text. You could add these new context variables as input_variables, or we can add a SimpleMemory to the chain to manage this context:



> 比如你想添加  `日期` `时间`, `地点` 这个上下文 <br>
> 并使用生成的概要,创建`社交媒体的帖子`<br>
> `SimpleMemory` 来管理 `input_variables`, `input_variables`是用来增加 `上下文的`

In [16]:
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory

llm = OpenAI(temperature=0.7)
template = """You are a social media manager for a theater company.  Given the title of play, the era it is set in, the date,time and location, the synopsis of the play, and the review of the play, it is your job to write a social media post for that play.

Here is some context about the time and location of the play:
Date and Time: {time}
Location: {location}

Play Synopsis:
{synopsis}
Review from a New York Times play critic of the above play:
{review}

Social Media Post:
"""
prompt_template = PromptTemplate(
    input_variables=["synopsis", "review", "time", "location"], template=template
)
social_chain = LLMChain(llm=llm, prompt=prompt_template, output_key="social_post_text")

overall_chain = SequentialChain(
    memory=SimpleMemory(
        memories={"time": "December 25th, 8pm PST", "location": "Theater in the Park"}
    ),
    chains=[synopsis_chain, review_chain, social_chain],
    input_variables=["era", "title"],
    # Here we return multiple variables
    output_variables=["social_post_text"],
    verbose=True,
)

overall_chain({"title": "Tragedy at sunset on the beach", "era": "Victorian England"})



> Entering new SequentialChain chain...

> Finished chain.


{'title': 'Tragedy at sunset on the beach',
 'era': 'Victorian England',
 'time': 'December 25th, 8pm PST',
 'location': 'Theater in the Park',
 'social_post_text': 'The love of a lifetime awaits you at @TheaterInThePark! 💜 Tragedy at Sunset on the Beach is a powerful and heartbreaking story of a young couple’s struggle against all odds to be together. Don’t miss this timeless and timely play on December 25th at 8pm PST! #TragedyAtSunset #LoveConquersAll #TheaterInThePark'}